# Практическое задание к уроку 4.
## Система управления базами данных MongoDB в Python

### Задание 1.
**Развернуть у себя на компьютере MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.**



In [14]:
# Установим mongoDB сервер и клиент
!apt install mongodb > log

"apt" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [15]:
!service mongodb start

"service" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [16]:
!pip install pymongo

In [93]:
# Подключаем необходимые модули и библиотеки
from pymongo import MongoClient
from pymongo.errors import InvalidDocument as idoc
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import json
from time import sleep
import pandas as pd

In [34]:
# будем парсить вакансии Дата-сайентиста

url = f"https://hh.ru/vacancies/data-scientist"
HEADERS = {
   
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

In [131]:

vacancies_list = []
i = 0
while i < 100: #Устанавливаем заведомо большое число страниц. Лишние отсеются по условию response.status_code != 200:
    sleep(0.1) #Устанавливаем искуственную задержку для исключения блокировки сайтом 
    url = 'https://hh.ru/vacancies/data-scientist?page=' + str(i)
    #url = 'https://hh.ru/vacancies/data-engineer?page=' + str(i)
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
      print('Парсинг завершен')
      break
    else:
        soup = bs(response.text, 'html.parser')
        vacancies = soup.find_all('div', {'class': 'serp-item'})
        #print(f'PAGE #{i}')   
        
        for item in vacancies:
            vacancies_list.append(
                {
         # На каждой странице определяем необходимые параметры                  
                    'title': item.find('a', {'class': 'serp-item__title'}).text, # Название вакансии
                    'link': item.find('a', {'class': 'serp-item__title'})['href'], # Ссылка на вакансию
                    'salary': item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}), # Зарплата
                    'organization': item.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text, # Название компании
                    'vacancy_source': 'hh.ru',
                    
                }
            )
    i = i + 1
vacancies_list
 

Парсинг завершен


[{'title': 'Data scientist',
  'link': 'https://hh.ru/vacancy/74376536?query=data+scientist',
  'salary': <span class="bloko-header-section-3" data-qa="vacancy-serp__vacancy-compensation">200 000 – 240 000 <!-- -->руб.</span>,
  'organization': 'Астор',
  'vacancy_source': 'hh.ru'},
 {'title': 'Data Scientist',
  'link': 'https://hh.ru/vacancy/76925483?query=data+scientist',
  'salary': <span class="bloko-header-section-3" data-qa="vacancy-serp__vacancy-compensation">160 000 – 200 000 <!-- -->руб.</span>,
  'organization': '7RedLines',
  'vacancy_source': 'hh.ru'},
 {'title': 'Data Scientist middle+/Senior',
  'link': 'https://hh.ru/vacancy/77208642?query=data+scientist',
  'salary': <span class="bloko-header-section-3" data-qa="vacancy-serp__vacancy-compensation">250 000 – 350 000 <!-- -->руб.</span>,
  'organization': 'ИП\xa0Мамаева Ирина Анатольевна',
  'vacancy_source': 'hh.ru'},
 {'title': 'Data Scientist (Middle+)',
  'link': 'https://hh.ru/vacancy/76810036?query=data+scientist',

In [132]:
 # Приведем данные к читаемому виду
for i in vacancies_list:
        # Link
        i['link'] =  i['link'].replace('?query=data+scientist', '') 
        
        # Organization
        i['organization'] = i['organization'].replace(u'\xa0', u' ')
        
        # Salary
        try:
            i['salary'] = i['salary'].text
        except:
            i['salary'] = None
        if i['salary']:
            # print(i['salary'])
            salary_list = i['salary'].split(' ')
            # print(salary_list)
            if salary_list[0] == 'от':
                i['salary_min'] = float(salary_list[1].replace('\u202f', ''))
                i['salary_max'] = None
            elif salary_list[0] == 'до':
                i['salary_min'] = None
                i['salary_max'] = float(salary_list[1].replace('\u202f', ''))
            else:
                i['salary_min'] = float(salary_list[0].replace('\u202f', ''))
                i['salary_max'] = float(salary_list[2].replace('\u202f', ''))
            i['salary_currency'] = salary_list[-1]
        else:
            i['salary_min'] = None
            i['salary_max'] = None
            i['salary_currency'] = None
        i.pop('salary')
        
vacancies_list


[{'title': 'Data scientist',
  'link': 'https://hh.ru/vacancy/74376536',
  'organization': 'Астор',
  'vacancy_source': 'hh.ru',
  'salary_min': 200000.0,
  'salary_max': 240000.0,
  'salary_currency': 'руб.'},
 {'title': 'Data Scientist',
  'link': 'https://hh.ru/vacancy/76925483',
  'organization': '7RedLines',
  'vacancy_source': 'hh.ru',
  'salary_min': 160000.0,
  'salary_max': 200000.0,
  'salary_currency': 'руб.'},
 {'title': 'Data Scientist middle+/Senior',
  'link': 'https://hh.ru/vacancy/77208642',
  'organization': 'ИП Мамаева Ирина Анатольевна',
  'vacancy_source': 'hh.ru',
  'salary_min': 250000.0,
  'salary_max': 350000.0,
  'salary_currency': 'руб.'},
 {'title': 'Data Scientist (Middle+)',
  'link': 'https://hh.ru/vacancy/76810036',
  'organization': 'DIGINETICA',
  'vacancy_source': 'hh.ru',
  'salary_min': 170000.0,
  'salary_max': 240000.0,
  'salary_currency': 'руб.'},
 {'title': 'Data scientist',
  'link': 'https://hh.ru/vacancy/74257661',
  'organization': 'НАО Фин

In [133]:
hh_result = pd.DataFrame(vacancies_list)
hh_result

,title,link,organization,vacancy_source,salary_min,salary_max,salary_currency
0,Data scientist,https://hh.ru/vacancy/74376536,Астор,hh.ru,200000.0,240000.0,руб.
1,Data Scientist,https://hh.ru/vacancy/76925483,7RedLines,hh.ru,160000.0,200000.0,руб.
2,Data Scientist middle+/Senior,https://hh.ru/vacancy/77208642,ИП Мамаева Ирина Анатольевна,hh.ru,250000.0,350000.0,руб.
3,Data Scientist (Middle+),https://hh.ru/vacancy/76810036,DIGINETICA,hh.ru,170000.0,240000.0,руб.
4,Data scientist,https://hh.ru/vacancy/74257661,НАО Финансовые системы,hh.ru,150000.0,300000.0,руб.
...,...,...,...,...,...,...,...
252,Commercial Department Assistant,https://hh.ru/vacancy/77294659,NielsenIQ,hh.ru,NaN,NaN,None
253,Старший инженер по внедрению (SberWorks),https://hh.ru/vacancy/76881257,Сбер для экспертов,hh.ru,NaN,NaN,None
254,Frontend Developer (Vue),https://hh.ru/vacancy/76810414,"Hi, Rockits!",hh.ru,180000.0,NaN,руб.
255,Руководитель программы проектов (DWH и BI),https://hh.ru/vacancy/76800172,ООО ПСБ Лизинг,hh.ru,NaN,NaN,None


### Запись данных в базу

In [38]:
# Создаем клиента
client = MongoClient()


In [39]:
# Создаём ссылку на базу данных vacancies_DS_hh
db = client.vacancies_DS_hh

In [48]:
# С помощью метода insеrt_many() передадим в коллекцию cars наш массив вакансий:
db.vacancies_hh.insert_many(vacancies_list)

In [49]:
# Посмотрим все коллекции в базе данных с именем vacancies_DS_hh:
print(db.list_collection_names())

['vacancies_hh']


In [80]:
db.vacancies_hh.find()

In [143]:
for i in db.vacancies_hh.find():
    pprint(i)

{'_id': ObjectId('63f717b4162bc6174f44e10a'),
 'link': 'https://hh.ru/vacancy/74376536',
 'organization': 'Астор',
 'salary_currency': 'руб.',
 'salary_max': 240000.0,
 'salary_min': 200000.0,
 'title': 'Data scientist',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f717b4162bc6174f44e10b'),
 'link': 'https://hh.ru/vacancy/76925483',
 'organization': '7RedLines',
 'salary_currency': 'руб.',
 'salary_max': 200000.0,
 'salary_min': 160000.0,
 'title': 'Data Scientist',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f717b4162bc6174f44e10c'),
 'link': 'https://hh.ru/vacancy/77208642',
 'organization': 'ИП Мамаева Ирина Анатольевна',
 'salary_currency': 'руб.',
 'salary_max': 350000.0,
 'salary_min': 250000.0,
 'title': 'Data Scientist middle+/Senior',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f717b4162bc6174f44e10d'),
 'link': 'https://hh.ru/vacancy/76810036',
 'organization': 'DIGINETICA',
 'salary_currency': 'руб.',
 'salary_max': 240000.0,
 'salary_min': 170000.0,
 'title': 'Da

In [134]:
# Для обновления отдельных полей в этих функциях применяется оператор $set. 
# Если документ не содержит обновляемое поле, то оно создается.
def insert_new_data(vacancies_list):
    for vac in vacancies_list:
        try:
            db.vacancies_hh.update_one({'link': vac['link']}, {'$set': vac}, upsert=True)
        except idoc:
            print(f'Error {idoc}')
            continue

In [135]:
# Проводим обновление вакансий
insert_new_data(vacancies_list)

In [136]:
number_vacancies = len(list(db.vacancies_hh.find()))
print("Cейчас на сайте 'hh.ru' %d вакансий по направлению 'Data scientist'" %number_vacancies)

Cейчас на сайте 'hh.ru' 263 вакансий по направлению 'Data scientist'


### Задание 2.
**Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).**



In [139]:
# Массив всех вакансий по направлению 'Data scientist' на сайте 'hh.ru' в настоящее время
v = db.vacancies_hh.find()

In [140]:
# Проверяем наличие
for i in v:
    pprint(i)

{'_id': ObjectId('63f717b4162bc6174f44e10a'),
 'link': 'https://hh.ru/vacancy/74376536',
 'organization': 'Астор',
 'salary_currency': 'руб.',
 'salary_max': 240000.0,
 'salary_min': 200000.0,
 'title': 'Data scientist',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f717b4162bc6174f44e10b'),
 'link': 'https://hh.ru/vacancy/76925483',
 'organization': '7RedLines',
 'salary_currency': 'руб.',
 'salary_max': 200000.0,
 'salary_min': 160000.0,
 'title': 'Data Scientist',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f717b4162bc6174f44e10c'),
 'link': 'https://hh.ru/vacancy/77208642',
 'organization': 'ИП Мамаева Ирина Анатольевна',
 'salary_currency': 'руб.',
 'salary_max': 350000.0,
 'salary_min': 250000.0,
 'title': 'Data Scientist middle+/Senior',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f717b4162bc6174f44e10d'),
 'link': 'https://hh.ru/vacancy/76810036',
 'organization': 'DIGINETICA',
 'salary_currency': 'руб.',
 'salary_max': 240000.0,
 'salary_min': 170000.0,
 'title': 'Da

In [142]:
#  Функция, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы
def search_by_salary():
    value = int(input('Введите минимальное требуемое значение зарплаты В РУБЛЯХ: '))
    kurs = float(input('Введите курс ОДНОГО доллара США В РУБЛЯХ: '))
    res = db.vacancies_hh.find({'$or':
                              [{'salary_currency': 'руб.',
                                    '$or': [
                                        {'salary_min': {'$gt': value}},
                                        {'salary_max': {'$gt': value}},]
                                },
                                {'salary_currency': 'USD',
                                    '$or': [
                                        {'salary_min': {'$gt': value / kurs}},
                                        {'salary_max': {'$gt': value / kurs}},]
                                }]
                            })

    return list(res)

res = search_by_salary()
pprint(res)


Введите минимальное требуемое значение зарплаты В РУБЛЯХ: 100000
Введите курс ОДНОГО доллара США В РУБЛЯХ: 74.89
[{'_id': ObjectId('63f717b4162bc6174f44e10a'),
  'link': 'https://hh.ru/vacancy/74376536',
  'organization': 'Астор',
  'salary_currency': 'руб.',
  'salary_max': 240000.0,
  'salary_min': 200000.0,
  'title': 'Data scientist',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('63f717b4162bc6174f44e10b'),
  'link': 'https://hh.ru/vacancy/76925483',
  'organization': '7RedLines',
  'salary_currency': 'руб.',
  'salary_max': 200000.0,
  'salary_min': 160000.0,
  'title': 'Data Scientist',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('63f717b4162bc6174f44e10c'),
  'link': 'https://hh.ru/vacancy/77208642',
  'organization': 'ИП Мамаева Ирина Анатольевна',
  'salary_currency': 'руб.',
  'salary_max': 350000.0,
  'salary_min': 250000.0,
  'title': 'Data Scientist middle+/Senior',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('63f717b4162bc6174f44e10d'),
  'link': 'https://hh.ru

In [144]:
hh_sal = pd.DataFrame(res)
hh_sal


,_id,link,organization,salary_currency,salary_max,salary_min,title,vacancy_source
0,63f717b4162bc6174f44e10a,https://hh.ru/vacancy/74376536,Астор,руб.,240000.0,200000.0,Data scientist,hh.ru
1,63f717b4162bc6174f44e10b,https://hh.ru/vacancy/76925483,7RedLines,руб.,200000.0,160000.0,Data Scientist,hh.ru
2,63f717b4162bc6174f44e10c,https://hh.ru/vacancy/77208642,ИП Мамаева Ирина Анатольевна,руб.,350000.0,250000.0,Data Scientist middle+/Senior,hh.ru
3,63f717b4162bc6174f44e10d,https://hh.ru/vacancy/76810036,DIGINETICA,руб.,240000.0,170000.0,Data Scientist (Middle+),hh.ru
4,63f717b4162bc6174f44e10e,https://hh.ru/vacancy/74257661,НАО Финансовые системы,руб.,300000.0,150000.0,Data scientist,hh.ru
5,63f717b4162bc6174f44e113,https://hh.ru/vacancy/70163749,Rubbles,руб.,400000.0,NaN,Data Scientist (middle/senior),hh.ru
6,63f717b4162bc6174f44e115,https://hh.ru/vacancy/77181469,ООО МФК ВЭББАНКИР,руб.,300000.0,120000.0,Data scientist / Аналитик риск-моделей,hh.ru
7,63f717b4162bc6174f44e11a,https://hh.ru/vacancy/73387970,Райффайзен Банк,руб.,350000.0,NaN,Senior Data Scientist,hh.ru
8,63f717b4162bc6174f44e132,https://hh.ru/vacancy/76955992,Kalinka - Realty,руб.,NaN,150000.0,Аналитик данных/Data Scientist,hh.ru
9,63f717b4162bc6174f44e142,https://hh.ru/vacancy/76674451,ITSS,USD,10000.0,4000.0,Data scientist (math modeling),hh.ru


In [145]:
# Cохраняем датафрейм в файл с учетом кодировки для кирилицы
hh_sal.to_csv('hh_vacancy_salary_DS.csv', index=False, encoding='utf-16')